In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
import scipy.io
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np 
from typing import Literal
import torch 
from torch import nn
from transphorm.model_components import AATrialDataModule, TrialClassifer, LSTMClassifer
from dotenv import load_dotenv
import os
load_dotenv()
data_path = Path(os.getenv('TRIAL_DATA_PATH'))
model_path = "/Users/mds8301/Development/transphorm/models/aa-classifiers/a0383409f8fe4c3683f83cda885528e0/checkpoints/trial_lstm_clf_v2.ckpt"

In [6]:
input_size = 6104
hidden_size = 8
num_lay = 2

class LSTMClassifer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout1d(.2)

        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # init hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros_like(h0).to(x.device)

        x_1, (h0, c0) = self.lstm(x, (h0, c0))

        # batch_norm = self.batch_norm(h0[-1])
        # dropout = self.dropout(batch_norm)
        # out = self.fc(dropout)

        return x_1, (h0, c0)
model = LSTMClassifer(input_size, hidden_size, num_lay)

In [7]:
data_mod = AATrialDataModule(data_path)
data_mod.prepare_data()
data_mod.setup('train')

In [10]:
test = data_mod.train[0:32][0]
x_1,(h0, c0) = model(test)

In [18]:
x_1[:, -1, :].shape

torch.Size([32, 8])

In [24]:
h0[-1].shape

torch.Size([32, 8])